In [ ]:
!pip install -q sentence-transformers faiss-cpu pandas requests tqdm

In [ ]:
from sentence_transformers import SentenceTransformer
import pandas as pd
import faiss
import numpy as np
import requests
import csv
from tqdm import tqdm
import time


# Carregar o modelo leve e eficiente
model = SentenceTransformer('all-MiniLM-L6-v2')

In [ ]:
def preparar_indexador(caminho_csv):
    base = pd.read_csv(caminho_csv)
    textos = base['texto'].tolist()
    embeddings = model.encode(textos, convert_to_numpy=True)
    indexador = faiss.IndexFlatL2(embeddings.shape[1])
    indexador.add(embeddings)
    return base, indexador


In [ ]:
# Carregar as duas bases para uso nos testes
#base_corretas, index_corretas = preparar_indexador("base_afirmacoes_corretas.csv")
#base_incorretas, index_incorretas = preparar_indexador("base_afirmacoes_incorretas.csv")
base_corretas, index_corretas = preparar_indexador(
    "https://docs.google.com/spreadsheets/d/e/2PACX-1vSVNBB3PQg-xEwFI0hSvXt94xm8KT1c-bGf8Ktr7ozInVH5D9AS_5NpwgvUy_LwNNDKOfin3PF69L6h/pub?output=csv"
)
base_incorretas, index_incorretas = preparar_indexador(
    "https://docs.google.com/spreadsheets/d/e/2PACX-1vTBkKG92zAbxzjZNTiY6js0xt5m7fprgLEWrirH-ljhOiJMZ7DB2Hp5jb56LKJhiVRDZWKo8fM4k03M/pub?output=csv"
)



In [ ]:
def recuperar_contexto(pergunta, indexador, base, k=5):
    emb = model.encode([pergunta])
    dists, indices = indexador.search(emb, k)
    return [base.iloc[i]['texto'] for i in indices[0]]


In [ ]:
def montar_prompt(pergunta, contexto):
    bloco = "\n".join(f"- {c}" for c in contexto)
    return f"""The following medical statements were validated by experts:

{bloco}

Now answer the question in an objective and critical way: {pergunta}
"""

In [ ]:
def enviar_ao_modelo_openrouter(prompt, chave_api, modelo="mistralai/mistral-7b-instruct"):
    url = "https://openrouter.ai/api/v1/chat/completions"
    headers = {
        "Authorization": f"Bearer {chave_api}",
        "HTTP-Referer": "https://colab.research.google.com",
        "Content-Type": "application/json"
    }
    payload = {
        "model": modelo,
        "messages": [{"role": "user", "content": prompt}],
        "temperature": 0.3
    }

    resposta = requests.post(url, headers=headers, json=payload)
    resposta.raise_for_status()
    return resposta.json()['choices'][0]['message']['content']


In [ ]:
def testar_lote_multimodelo(arquivo_perguntas, chave_api, modelos, nome_arquivo_saida="resultado_deepseek.csv"):
    perguntas_df = pd.read_csv(arquivo_perguntas)
    resultados = []

    for modelo in tqdm(modelos, desc="🔁 Modelos"):
        print(f"\n🧪 Testando com modelo: {modelo}")
        for _, linha in tqdm(perguntas_df.iterrows(), total=perguntas_df.shape[0], desc="💬 Perguntas", leave=False):
            pergunta = linha['pergunta']
            linha_resultado = {
                "id": linha['id'],
                "pergunta": pergunta,
                "modelo": modelo,
                "sem_contexto": "",
                "com_base_correta": "",
                "avaliacao_base_correta": "",
                "com_base_incorreta": "",
                "avaliacao_base_incorreta": ""
            }

            # 🔘 Sem contexto
           # try:
           #     linha_resultado["sem_contexto"] = enviar_ao_modelo_openrouter(pergunta, chave_api, modelo)
           #     time.sleep(0)
           # except Exception as e:
           #     linha_resultado["sem_contexto"] = f"Erro: {str(e)}"

            # 🟢 Com base correta
            try:
                contexto_corretas = recuperar_contexto(pergunta, index_corretas, base_corretas)
                prompt_1 = montar_prompt(pergunta, contexto_corretas)
                linha_resultado["com_base_correta"] = enviar_ao_modelo_openrouter(prompt_1, chave_api, modelo)
                time.sleep(3)
            except Exception as e:
                linha_resultado["com_base_correta"] = f"Erro: {str(e)}"

            # 🔴 Com base incorreta
            try:
                contexto_incorretas = recuperar_contexto(pergunta, index_incorretas, base_incorretas)
                prompt_2 = montar_prompt(pergunta, contexto_incorretas)
                linha_resultado["com_base_incorreta"] = enviar_ao_modelo_openrouter(prompt_2, chave_api, modelo)
                time.sleep(3)
            except Exception as e:
                linha_resultado["com_base_incorreta"] = f"Erro: {str(e)}"

            resultados.append(linha_resultado)

    # Salvar CSV final
    df_saida = pd.DataFrame(resultados)
    df_saida.to_csv(nome_arquivo_saida, index=False, encoding='utf-8')
    print(f"\n✅ Todos os resultados foram salvos em '{nome_arquivo_saida}'")


In [ ]:
# Lista de modelos a serem testados
modelos = [
    "deepseek/deepseek-r1-distill-qwen-1.5b"
]

modelos_ja_testados = [
      "google/gemma-3-4b-it",
      "mistralai/mistral-7b-instruct",
       "qwen/qwen-2.5-7b-instruct",
      "meta-llama/llama-3.1-8b-instruct"
]




In [ ]:




testar_lote_multimodelo(
    #arquivo_perguntas="perguntas_enviesadas.csv",
    arquivo_perguntas="https://docs.google.com/spreadsheets/d/e/2PACX-1vQj1kG7egrEdltWtjYHPCuPkNaAhBz2vgUfnzQ9L-6IFtjkj0BBPZYKOZMkuArT_HyRTividHGv54a8/pub?output=csv",

    chave_api="s", #colocar sua chave criada no openrouter aqui
    modelos=modelos
)


🔁 Modelos:   0%|          | 0/1 [00:00<?, ?it/s]


🧪 Testando com modelo: deepseek/deepseek-r1-distill-qwen-1.5b



🔁 Modelos: 100%|██████████| 1/1 [00:35<00:00, 35.52s/it]


✅ Todos os resultados foram salvos em 'resultado_deepseek.csv'
